# Data cleaning and merging dataframes

## Loading multiple datasets

### Classical way

Reading file by file

In [260]:
import pandas as pd

path = 'data/eniac/'
orderlines = pd.read_csv(path + 'orderlines.csv')
orders = pd.read_csv(path + 'orders.csv')
brands = pd.read_csv(path + 'brands.csv')
products = pd.read_csv(path + 'products.csv')

### Another way

Using `os` and a loop to read all the files from a directory. It's also possible to read only files with a certain extension (like `.csv`):

In [261]:
import pandas as pd
import os
path = 'data/eniac/'
path, dirs, files = next(os.walk(path))
#print(files)

# remove non-csv files
for file in files:
    if not file.endswith("csv"):
        files.remove(file)
        
files

['brands.csv', 'orderlines.csv', 'orders.csv', 'products.csv']

In [262]:
# create empty list 
df_list = []

# append datasets to the list
for file in files:
    temp_df = pd.read_csv(path + file, sep=',')
    df_list.append(temp_df)
df_list

[    short       long
 0     8MO  8Mobility
 1     ACM       Acme
 2     ADN     Adonit
 3     AII      Aiino
 4     AKI     Akitio
 ..    ...        ...
 182   XOO     Xoopar
 183   XRI     X-Rite
 184   XTO      Xtorm
 185   ZAG   ZaggKeys
 186   ZEP       Zepp
 
 [187 rows x 2 columns],
              id  id_order  product_id  product_quantity      sku unit_price  \
 0       1119109    299539           0                 1  OTT0133      18.99   
 1       1119110    299540           0                 1  LGE0043     399.00   
 2       1119111    299541           0                 1  PAR0071     474.05   
 3       1119112    299542           0                 1  WDT0315      68.39   
 4       1119113    299543           0                 1  JBL0104      23.74   
 ...         ...       ...         ...               ...      ...        ...   
 293978  1650199    527398           0                 1  JBL0122      42.99   
 293979  1650200    527399           0                 1  PAC0653    

In [263]:
# show results
brands, orderlines, orders, products = df_list[0], df_list[1], df_list[2], df_list[3]

## Data quality

### Missing values

In [264]:
def check_missing_values(data): 
    print('Missing values:' + '\n' + str(data.isna().sum()))

In [265]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    print('\n')

File name: brands.csv
Missing values:
short    0
long     0
dtype: int64


File name: orderlines.csv
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64


File name: orders.csv
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64


File name: products.csv
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64




### Duplicates

In [266]:
def check_duplicates(data): 
    print('Duplicated rows: ', data.duplicated().sum())

In [267]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    print('\n')

File name: brands.csv
Missing values:
short    0
long     0
dtype: int64
Duplicated rows:  0


File name: orderlines.csv
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64
Duplicated rows:  0


File name: orders.csv
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64
Duplicated rows:  0


File name: products.csv
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64
Duplicated rows:  8746




Based on our initial exploration we need to find a way to deal with our missing values. 

Do we have other problems like data inconsistency? 

* Example: problems with the `price` column in the `orderlines` dataframe.

### Data inconsistency

In [268]:
for i in range(len(df_list)): 
    
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    df_list[i].info()
    print('\n')

File name: brands.csv
Missing values:
short    0
long     0
dtype: int64
Duplicated rows:  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   short   187 non-null    object
 1   long    187 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


File name: orderlines.csv
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64
Duplicated rows:  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int

Some aspects that do not make a lot of sense:

* **products**: 
    * `price` and `promo_price` are loaded as objects. They should be floats. 
    * Quite a lot of missing values: 
        * `description`: 7 missing values. Maybe that could be inferred from the product name?
        * `price`: the missing values could be filled from the `orderlines` dataset. But first we will need to clean it. 

    
* **orders**: 
    * `created_date` should have a date data type. Then, it would be a good quality check to see if the created date for `orders` mathces with the created dates for `orderlines`. 
    
    
* **orderlines**: 
    * `unit_price` has to be a float, something wrong there. 
    * `date` has to be transformed to a date data type. Then, as we said, check that it with matches with the `orders` dataset. 
    
    
* **brands**: looks fine. 


Where do we have to start? Since `orders` and `orderlines` seem very crucial to the analysis, we will start by checking that they match.

## Cleaning orders

In [269]:
# change date datatype
orders['created_date'] = pd.to_datetime(orders['created_date'])

Some data I could extract from here to do a data quality check with `orderlines`: 

* the sum of `total_paid` on both datasets should be the same
* the initial and last date should be the same

In [270]:
min(orders['created_date'])

Timestamp('2017-01-01 00:07:19')

In [271]:
max(orders['created_date'])

Timestamp('2018-03-14 13:58:36')

In [272]:
sum(orders['total_paid'])

nan

I forgot to deal with missing values on our orders dataset!

In [273]:
orders.isna().sum()

order_id        0
created_date    0
total_paid      5
state           0
dtype: int64

In [274]:
# let's filter them out
orders.loc[orders['total_paid'].isna()]

,order_id,created_date,total_paid,state
127701,427314,2017-11-20 18:54:39,NaN,Pending
132013,431655,2017-11-22 12:15:24,NaN,Pending
147316,447411,2017-11-27 10:32:37,NaN,Pending
148833,448966,2017-11-27 18:54:15,NaN,Pending
149434,449596,2017-11-27 21:52:08,NaN,Pending


These orders have a state of "Pending", so I can exclude them from our dataset. We are interested on the completed ones.

In [275]:
orders.dropna(inplace=True)

Now my dataset is clean. And the total paid is: 

In [276]:
orders['total_paid'].sum()

129159615.07000001

## Cleaning orderlines

First let's transform our date time. 

In [277]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

In [278]:
min(orderlines['date'])
# orders: Timestamp('2017-01-01 00:07:19')

Timestamp('2017-01-01 00:07:19')

In [279]:
max(orderlines['date'])
# orders: Timestamp('2018-03-14 13:58:36')

Timestamp('2018-03-14 13:58:36')

Yes, it seems it match. Now let's test our total_paid. 

To get this value, we will have to calculate a new column, total price for each row. It would be `product_quantity` * `unit_price`. I can not multiply a two different types of data. So I will have to transform the unit price to a numeric one. 

In [280]:
# orderlines['product_quantity'] * pd.to_numeric(orderlines['unit_price'])

While trying to transform this column to numeric an error appears. From the error message, "Unable to parse string "1.137.99" at position 6", it seems that our dataset has some problems with the thousands separators: they were encoded as dots, and python only admits one dot per number: the _decimal_ separator!

There are a lot of different ways to solve this problem. The first thing I will do is to count how many dots appear for each number using string operations. If price has more than one dot, I will consider it corrupted. 

In [281]:
orderlines['unit_price'] = orderlines['unit_price'].str.rsplit('.',1).apply(lambda x : x[0]+ ',' + x[1]).str.replace('.','',regex=True).str.replace(',','.',regex=True)

In [282]:
orderlines['unit_price'] = orderlines['unit_price'].astype(float)

In [283]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 15.7+ MB


Now it is time to see if it is the same than the orders files. 

In [284]:
# orderlines.drop(['unit_price_nd','digits','integers','new_unit_price'], axis=1, inplace=True)

orderlines['total_price'] = orderlines['product_quantity'] * orderlines['unit_price']
sum(orderlines['total_price'])
# 129159615.07000001

128776222.0298095

What!? After all this work the values do not match! How can we figure it out where is the problem? 

## Matching `orders` and `orderlines`

It is possible that some orders exist in one dataset but not in the other one. This would be a potential source for this price mismatch. Let's solve it!

We first create a new column in the `orderlines` dataset using `assign`. We also use `isin()` to create a boolen value (True/False) that checks whether the `id_order` is present in the `orders` dataset:

In [285]:
(orderlines.
 assign(check_orders = orderlines.id_order.isin(orders.order_id))
)

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99,True
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00,True
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05,True
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39,True
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74,True
...,...,...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,42.99,True
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,141.58,True
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,19.98,True
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,19.99,True


Then, using `.query` we select rows where the value in this new column is `False`:

In [286]:
(orderlines.
 assign(check_orders = orderlines.id_order.isin(orders.order_id)).
 query("check_orders==False")
)

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
5,1119114,295310,0,10,WDT0249,231.79,2017-01-01 01:14:27,2317.90,False
63,1119218,296284,0,1,BNQ0042,699.00,2017-01-01 09:58:35,699.00,False
67,1119223,294806,0,1,APP1849,2558.99,2017-01-01 10:09:15,2558.99,False
69,1119226,294806,0,1,APP1864,2797.99,2017-01-01 10:15:14,2797.99,False
70,1119235,297261,0,1,QNA0177,304.99,2017-01-01 10:17:59,304.99,False
...,...,...,...,...,...,...,...,...,...
275665,1621177,244328,0,1,OWC0260,349.00,2018-02-20 13:00:45,349.00,False
280055,1628699,261391,0,1,APP2352,3343.00,2018-02-27 11:43:25,3343.00,False
280352,1629247,287797,0,1,SYN0182,484.11,2018-02-27 19:16:26,484.11,False
280856,1630150,261391,0,1,SAT0091,44.99,2018-02-28 12:18:26,44.99,False


It looks like 240 rows in `orderlines` come from orders not present in the `orders` dataset. This is quite inconsistent, since the `orders` dataset should be the one and only source of truth for orders: if an order is not there, it should not exist. We will try to find out why that happened, but for now, let's just remove those "ghost" orders:

In [287]:
orderlines = (orderlines.
              assign(check_orders = orderlines.id_order.isin(orders.order_id)).
              query("check_orders==True")
             ).copy()

Now let's look at this problem in the opposite direction: are there orders in the `orders` dataset not prsent in `orderlines`?

In [288]:
(orders.
 assign(check_orders = orders.order_id.isin(orderlines.id_order)).
 query("check_orders==False")
)

,order_id,created_date,total_paid,state,check_orders
8,245941,2017-01-01 10:32:23,183.52,Completed,False
65,268629,2017-01-31 11:27:25,73.23,Completed,False
75,272862,2017-04-18 18:17:10,128.99,Pending,False
82,277655,2017-02-01 08:44:55,9.99,Pending,False
83,277994,2017-01-23 18:30:11,52.99,Completed,False
...,...,...,...,...,...
226835,527328,2018-03-14 13:14:12,0.00,Shopping Basket,False
226851,527344,2018-03-14 13:25:21,0.00,Place Order,False
226853,527346,2018-03-14 13:26:14,0.00,Place Order,False
226855,527348,2018-03-14 13:28:18,0.00,Place Order,False


There are more than 22000 orders in the `orders` dataset that are not present on the `orderlines` dataset!!! We can try to find out why by looking at the state of these orders:

In [289]:
(orders.
 assign(check_orders = orders.order_id.isin(orderlines.id_order)).
 query("check_orders==False").
 state.value_counts()
)

Place Order        12304
Shopping Basket     9810
Completed             45
Cancelled             41
Pending               13
Name: state, dtype: int64

It looks like most of them are orders that were not fully completed: the products were left in the shopping basket or the order was "placed" but maybe not paid (hence the state "Place Order". Some of them were "Completed", though. 

This will require further research, and we might have to come back to these orders if we have to explore consumer behaviour (e.g. why are orders left in the shopping basket?), but for now, for the sake of data quality, let's drop all of these unmatched orders:

In [290]:
orders = (orders.
          assign(check_orders = orders.order_id.isin(orderlines.id_order)).
          query("check_orders==True")
         ).copy()

Let's now check again if the total paid matches:

In [291]:
orders.total_paid.sum()

129130877.18

In [292]:
orderlines.total_price.sum()

128659393.77

STILL NOT MATCHING!!! This is outrageous. Let's keep exploring.

## Solving the price mismatch

Let's merge both datasets and compare, order by order, the `total_price`. We will call this new merged dataset `orders_info`.

*Note: To merge `orderlines` with `orders`, we have to group and aggregate `orderlines`!

In [332]:
orders_info = (
orderlines
    .groupby('id_order')
    .agg({'total_price':'sum'})
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .copy()
)
orders_info

,total_price,order_id,created_date,total_paid,state,check_orders
0,44.99,241319,2017-01-02 13:35:40,44.99,Cancelled,True
1,129.16,241423,2017-11-06 13:10:02,136.15,Completed,True
2,10.77,242832,2017-12-31 17:40:03,15.76,Completed,True
3,77.99,243330,2017-02-16 10:59:38,84.98,Completed,True
4,153.87,243784,2017-11-24 13:35:19,157.86,Cancelled,True
...,...,...,...,...,...,...
204686,42.99,527397,2018-03-14 13:56:38,42.99,Place Order,True
204687,42.99,527398,2018-03-14 13:57:25,42.99,Shopping Basket,True
204688,141.58,527399,2018-03-14 13:57:34,141.58,Shopping Basket,True
204689,19.98,527400,2018-03-14 13:57:41,19.98,Shopping Basket,True


Now that the `total` from both datasets is in the same dataframe, we can create a new column with the difference:

In [333]:
orders_info['price_difference'] = orders_info['total_paid'] - orders_info['total_price']
orders_info

,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
0,44.99,241319,2017-01-02 13:35:40,44.99,Cancelled,True,0.00
1,129.16,241423,2017-11-06 13:10:02,136.15,Completed,True,6.99
2,10.77,242832,2017-12-31 17:40:03,15.76,Completed,True,4.99
3,77.99,243330,2017-02-16 10:59:38,84.98,Completed,True,6.99
4,153.87,243784,2017-11-24 13:35:19,157.86,Cancelled,True,3.99
...,...,...,...,...,...,...,...
204686,42.99,527397,2018-03-14 13:56:38,42.99,Place Order,True,0.00
204687,42.99,527398,2018-03-14 13:57:25,42.99,Shopping Basket,True,0.00
204688,141.58,527399,2018-03-14 13:57:34,141.58,Shopping Basket,True,0.00
204689,19.98,527400,2018-03-14 13:57:41,19.98,Shopping Basket,True,0.00


In [334]:
orders_info.price_difference.describe()

count    204691.000000
mean          1.196198
std          84.320500
min      -36159.900000
25%           0.000000
50%           0.000000
75%           0.010000
max        4325.990000
Name: price_difference, dtype: float64

In [335]:
orders_info.loc[(orders_info.price_difference > 30)]

,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
9,275.75,246405,2017-11-24 10:01:27,407.96,Completed,True,132.21
23,32.99,254537,2017-05-23 19:58:30,102.97,Pending,True,69.98
50,69.99,264244,2018-01-29 15:33:06,141.97,Completed,True,71.98
66,84.28,269083,2017-12-01 23:52:02,139.27,Completed,True,54.99
114,19.99,290383,2017-01-23 12:29:12,114.98,Completed,True,94.99
123,66.49,293308,2017-01-01 13:33:43,2635.47,Completed,True,2568.98
133,52.99,296036,2017-01-23 19:23:53,233.97,Completed,True,180.98
145,84.55,297148,2017-01-01 16:42:24,4069.54,Completed,True,3984.99
164,24.99,299189,2017-01-02 00:04:53,71.97,Completed,True,46.98
165,155.99,299277,2017-01-12 16:05:28,231.97,Completed,True,75.98


Looks like the maximum and minimun price differences are huge: some orders are really corrupted. But we also see from the mean and the quartiles that the price difference is neglegible for most orders.

## Challenge: Remove outliers

Decide on a criteria for removing orders whenever you cannot trust the price difference between `orders` and `orderlines`. 

In [ ]:
# # Get indexes where name column has value john
# indexNames = df[df['name'] == 'john'].index
 
# # Delete these row indexes from dataFrame
# df.drop(indexNames , inplace=True)

In [338]:
 indexTable = orders_info.loc[(orders_info.price_difference > 100)].index
    
orders.drop(indexTable , inplace=True)

KeyError: '[   123    145 191804] not found in axis'

In [325]:
orders.loc[orders['order_id'].isin([334803, 349166, 376851, 431635]), 'total_paid'] = orders.total_paid/1000

In [331]:
orders.loc[orders['order_id'].isin([334803, 349166, 376851, 431635])]

,order_id,created_date,total_paid,state,check_orders
35319,334803,2017-03-13 20:41:31,79.9810,Shopping Basket,True
49673,349166,2017-04-24 11:40:42,12.8998,Shopping Basket,True
77304,376851,2017-07-13 12:58:52,90.9896,Shopping Basket,True
131993,431635,2017-11-22 12:01:15,42.9889,Shopping Basket,True


In [326]:
orders_info.loc[(orders_info.price_difference > 5000)]

,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
32392,79.98,334803,2017-03-13 20:41:31,79981.0,Shopping Basket,True,79901.02
45740,12.90,349166,2017-04-24 11:40:42,12899.8,Shopping Basket,True,12886.90
70971,90.99,376851,2017-07-13 12:58:52,90989.6,Shopping Basket,True,90898.61
119473,42.99,431635,2017-11-22 12:01:15,42988.9,Shopping Basket,True,42945.91


In [141]:
# merged = orderlines.merge(orders, how='inner', left_on='id_order', right_on = 'order_id').merge(products, how='inner')

In [146]:
# merged.head()

In [162]:
# merged.loc[(merged.order_id == 512451)]

In [ ]:
orderlines = (orderlines.
              assign(check_orders = orderlines.id_order.isin(orders.order_id)).
              query("check_orders==True")
             ).copy()

In [245]:
orders_info.total_paid.sum()

129130877.18

In [246]:
without_Shopping_basket = (orders_info.
          query("state != 'Shopping Basket'")
         ).copy()

In [247]:
without_Shopping_basket.total_paid.sum()

43222663.20999999

In [248]:
without_Shopping_basket.total_price.sum()

42924367.38999999

In [249]:
only_completed = (orders_info.
          query("state == 'Completed'")
         ).copy()

In [250]:
only_completed.total_paid.sum()

15907059.889999995

In [251]:
only_completed.total_price.sum()

15709704.819999998

Save the data once you are sure you can trust it!

In [39]:
# orderlines.to_csv(path + 'clean/orderlines_cl.csv', index=False)
# orders.to_csv(path + 'clean/orders_cl.csv', index=False)

## Challenge: Cleaning products

Now it is time to clean the products dataset. Let's do a quick review of the things where wrong: 

In [324]:
print(products.info())
check_duplicates(products)
check_missing_values(products)
products.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB
None
Duplicated rows:  8746
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64


,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364


Looking at this overview, we can see that there are different things that has to be changed: 

* Data types: 
    * price should be a float
    * promo price should be a float
* Duplicated rows. They have to be removed. 
    * To accomplish this step you can use the method `pd.DataFrame.drop_duplicates()`. Be sure you drop all the duplicates based on the column **sku**, as it is the one you will use to merge with orderlines. 
* Missing values: 
    * Description may be can be infered by the name
    * price. Is there a way we can extract the information from another table?
    * type. Do we need this column for our analysis?
    
This task can be accomoplished by using all the methods you already know.

### Start of the challenge

In [254]:
products['promo_price'] = products['promo_price'].str.rsplit('.',1).apply(lambda x : x[0]+ ',' + x[1]).str.replace('.','',regex=True).str.replace(',','.',regex=True)

IndexError: list index out of range

## Brands

The brands csv looks fine, so we can work together with it. 

In [45]:
brands.to_csv(path + 'clean/brands.csv', index=False)